In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import glob
import os

files = os.path.join("C:\\Users\\Adesh Suri\\Desktop\\angel", "bq-markov*.csv")

files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

df = pd.concat(map(pd.read_csv, files), ignore_index=True)

#df = pd.read_csv('bq-Markov1.csv')
df = df.sort_values(['visitStartTime'],
                    ascending=[True])
df['visit_order'] = df.groupby('AUC_Code').cumcount() + 1

In [ ]:
df

In [ ]:
cols = ['(not set)',
'google-play',
'(direct)',
'(not%20set)',
'google',
'youtube.com',
'web',
#'THTNO_Digital',
#'BNPFH_Digital',
#'SMS',
#'l.instagram.com',
#'sms',
#'stepupnetwork.in',
#'deals102',
#'menu',
#'OTOPI_Digital',
#'apps.facebook.com',
#'www-angelone-in.cdn.ampproject.org',
#'RnE'
]

df['source'].loc[df['source'].apply(lambda x: (x not in cols) )] = 'Others'


In [ ]:
df

In [ ]:
df_paths = df.groupby('AUC_Code')['source'].aggregate(
    lambda x: x.unique().tolist()).reset_index()
    
df_last_interaction = df.drop_duplicates('AUC_Code', keep='last')[['AUC_Code', 'Conversion']]
df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='AUC_Code')

df_paths['path'] = np.where( df_paths['Conversion'] == 0,
['Start, '] + df_paths['source'].apply(', '.join) + [', Null'],
['Start, '] + df_paths['source'].apply(', '.join) + [', Conversion'])


df_paths['path'] = df_paths['path'].str.split(', ')

df_paths = df_paths[['AUC_Code', 'path']]

In [ ]:
df_paths

In [ ]:
count=0
for path in df_paths['path']:
    for step in path:
        if step in ['Conversion']:
            count+=1
print (count)

In [ ]:
print(df_paths.loc[df_paths[df_paths['AUC_Code'] == 'AKC77313-0000'].index.to_numpy(),'path'])
print(df_paths.loc[df_paths[df_paths['AUC_Code'] == 'AJR91864-0000'].index.to_numpy(),'path'])

In [ ]:
list_of_paths = df_paths['path']
total_conversions = sum(path.count('Conversion') for path in df_paths['path'].tolist())
base_conversion_rate = total_conversions / len(list_of_paths)

In [ ]:
print(total_conversions)
print(base_conversion_rate)

In [ ]:
def transition_states(list_of_paths):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    transition_states = {x + '>' + y: 0 for x in list_of_unique_channels for y in list_of_unique_channels}
    print(len(list_of_unique_channels))
    print(list_of_unique_channels)
    for possible_state in list_of_unique_channels:
        if possible_state not in ['Conversion', 'Null']:
            for user_path in list_of_paths:
                if possible_state in user_path:
                    indices = [i for i, s in enumerate(user_path) if possible_state in s]
                    for col in indices:
                        transition_states[user_path[col] + '>' + user_path[col + 1]] += 1

    return transition_states


trans_states = transition_states(list_of_paths)

In [ ]:
print(len(trans_states))
print(trans_states)


In [ ]:
def transition_prob(trans_dict):
    list_of_unique_channels = set(x for element in list_of_paths for x in element)
    trans_prob = defaultdict(dict)
    for state in list_of_unique_channels:
        if state not in ['Conversion', 'Null']:
            counter = 0
            index = [i for i, s in enumerate(trans_dict) if state + '>' in s]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    counter += trans_dict[list(trans_dict)[col]]
            for col in index:
                if trans_dict[list(trans_dict)[col]] > 0:
                    state_prob = float((trans_dict[list(trans_dict)[col]])) / float(counter)
                    trans_prob[list(trans_dict)[col]] = state_prob

    return trans_prob


trans_prob = transition_prob(trans_states)

In [ ]:
trans_prob

In [ ]:
def transition_matrix(list_of_paths, transition_probabilities):
    trans_matrix = pd.DataFrame()
    list_of_unique_channels = set(x for element in list_of_paths for x in element)

    for channel in list_of_unique_channels:
        trans_matrix[channel] = 0.00
        trans_matrix.loc[channel] = 0.00
        trans_matrix.loc[channel][channel] = 1.0 if channel in ['Conversion', 'Null'] else 0.0

    for key, value in transition_probabilities.items():
        origin, destination = key.split('>')
        trans_matrix.at[origin, destination] = value

    return trans_matrix


trans_matrix = transition_matrix(list_of_paths, trans_prob)

In [ ]:
trans_matrix

In [ ]:
def removal_effects(df, conversion_rate):
    removal_effects_dict = {}
    channels = [channel for channel in df.columns if channel not in ['Start',
                                                                     'Null',
                                                                     'Conversion']]
    for channel in channels:
        removal_df = df.drop(channel, axis=1).drop(channel, axis=0)
        for column in removal_df.columns:
            row_sum = np.sum(list(removal_df.loc[column]))
            null_pct = float(1) - row_sum
            if null_pct != 0:
                removal_df.loc[column]['Null'] = null_pct
            removal_df.loc['Null']['Null'] = 1.0

        removal_to_conv = removal_df[
            ['Null', 'Conversion']].drop(['Null', 'Conversion'], axis=0)
        removal_to_non_conv = removal_df.drop(
            ['Null', 'Conversion'], axis=1).drop(['Null', 'Conversion'], axis=0)

        removal_inv_diff = np.linalg.inv(
            np.identity(
                len(removal_to_non_conv.columns)) - np.asarray(removal_to_non_conv))
        removal_dot_prod = np.dot(removal_inv_diff, np.asarray(removal_to_conv))
        removal_cvr = pd.DataFrame(removal_dot_prod,
                                   index=removal_to_conv.index)[[1]].loc['Start'].values[0]
        
        removal_effect = 1 - removal_cvr / conversion_rate
        removal_effects_dict[channel] = removal_effect

    return removal_effects_dict


removal_effects_dict = removal_effects(trans_matrix, base_conversion_rate)

In [ ]:
effect=dict(sorted(removal_effects_dict.items(), key=lambda kv:
                 (kv[1], kv[0]),reverse=True))
effect

In [ ]:
 def markov_chain_allocations(removal_effects, total_conversions):
    re_sum = np.sum(list(removal_effects.values()))

    return {k: (v / re_sum) * total_conversions for k, v in removal_effects.items()}




attributions = markov_chain_allocations(removal_effects_dict, total_conversions)

In [ ]:
attributions

In [ ]:
import matplotlib.pyplot as plt

names = list(effect.keys())
values = list(effect.values())

print(effect)
plt.figure(figsize=(10, 4))  # width:20, height:3
plt.bar(range(len(effect)), values, tick_label=names, align='edge', width=0.3)
plt.show()


In [ ]:

list_of_paths2= list(tuple(str(tuple(i)) for i in list_of_paths))
list_of_unique_paths=list(set(str(tuple(i)) for i in list_of_paths))
#list_of_unique_paths
freq_paths=dict()
for i in list_of_unique_paths:
    freq_paths[i]=0
for path in list_of_paths2:
    if path in freq_paths:
        freq_paths[path]+=1


In [ ]:
freq_paths=dict(sorted(freq_paths.items(), key=lambda item: item[1],reverse=True))
res = dict(list(freq_paths.items())[0: 50])

In [ ]:
res

In [ ]:
with open('path_freq.csv', 'w') as f:
    for key in res.keys():
        f.write("%s,%s\n"%(key,res[key]))

In [ ]:
count=0

for i in list_of_paths:
    for j in i:
        if j in ['Null'] and (len(i)==4):
            count+=1
print(count)

In [ ]:
count=0

for i in list_of_paths:
    if (len(i)>4):
            count+=1
print(count)